In [10]:
datasetname = "Vibration_dataset.csv"
datasetdir = "../datasets/vibration dataset/"
figuredir = "../figures/xgboost/"
import os

# You should change 'test' to your preferred folder.
CHECK_FOLDER = os.path.isdir(datasetdir)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(datasetdir)
    print("created folder : ", datasetdir)

else:
    print(datasetdir, "folder already exists.")
    
# You should change 'test' to your preferred folder.
CHECK_FOLDER = os.path.isdir(figuredir)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(figuredir)
    print("created folder : ", figuredir)

else:
    print(figuredir, "folder already exists.")

../datasets/vibration dataset/ folder already exists.
../figures/xgboost/ folder already exists.


In [11]:
import numpy as np
import pandas as pd
import seaborn as sns
#from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

#import torch
#import torch.nn as nn
#import torch.optim as optim
#from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [12]:
plt.rcParams['agg.path.chunksize'] = 100000
dpi = 1200

In [13]:
#df = pd.read_csv("../datasets/allfaultdatasetfiltered13.csv")
## to rename outerfault 
#df = pd.read_csv(datasetdir+datasetname)
#df = pd.read_csv("../datasets/allfaultdatasetfiltered13.csv")
## to rename outerfault 
df = pd.read_csv(datasetdir+datasetname, index_col ="class" )
"""Droping Series Column which is no longer required 
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html

df.drop(['B', 'C'], axis=1)

also rearrange columns order
df = df[['speed','vibration','Faults']]

#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html
#df.rename(columns={"A": "a", "B": "c"})
"""
df = df.drop(['series'], axis=1)
df = df.rename(index={'BallFault':'Ball'})
df = df.rename(index={'InnerFault':'Innerrace'})
df = df.rename(index={'outerFault':'Outerrace'})
df = df.reset_index()
df = df.rename(columns  ={'class':'Faults'})
df = df[['speed','vibration','Faults']]
df.head() , df['Faults'].unique()

(      speed  vibration Faults
 0  4.217042   0.003062   Ball
 1  4.219672   0.004049   Ball
 2  4.221645   0.001418   Ball
 3  0.036346  -0.002199   Ball
 4  0.036017  -0.003515   Ball,
 array(['Ball', 'Innerrace', 'Outerrace'], dtype=object))

In [14]:
newBallFault = df[df['Faults']=='Ball']
newBallFault['z_score'] = (newBallFault['vibration'] - newBallFault['vibration'].mean()) / newBallFault['vibration'].std()
newInnerFault = df[df['Faults']=='Innerrace']
newInnerFault['z_score'] = (newInnerFault['vibration'] - newInnerFault['vibration'].mean()) / newInnerFault['vibration'].std()
newouterFault = df[df['Faults']=='Outerrace']
newouterFault['z_score'] = (newouterFault['vibration'] - newouterFault['vibration'].mean()) / newouterFault['vibration'].std()

/tmp/ipykernel_414681/623848290.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newBallFault['z_score'] = (newBallFault['vibration'] - newBallFault['vibration'].mean()) / newBallFault['vibration'].std()
/tmp/ipykernel_414681/623848290.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newInnerFault['z_score'] = (newInnerFault['vibration'] - newInnerFault['vibration'].mean()) / newInnerFault['vibration'].std()
/tmp/ipykernel_414681/623848290.py:6: SettingWithCopyWarning: 
A value is trying to be set on 

In [15]:
new_df = newBallFault
new_df = new_df.append(newInnerFault)
new_df = new_df.append(newouterFault)

/tmp/ipykernel_414681/2952831548.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(newInnerFault)
/tmp/ipykernel_414681/2952831548.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(newouterFault)


In [16]:
new_df = new_df.drop_duplicates(subset = ["speed", 'vibration','z_score'])

In [17]:
df = new_df

In [18]:
df = df[['speed', 'vibration', 'z_score', 'Faults']]
df

,speed,vibration,z_score,Faults
0,4.217042,0.003062,0.291073,Ball
1,4.219672,0.004049,0.377935,Ball
2,4.221645,0.001418,0.146302,Ball
3,0.036346,-0.002199,-0.172193,Ball
4,0.036017,-0.003515,-0.288009,Ball
...,...,...,...,...
5667524,0.126758,0.010297,1.222905,Outerrace
5667542,3.131096,-0.001213,0.036795,Outerrace
5667560,1.329083,-0.000555,0.104573,Outerrace
5667571,3.977693,-0.000884,0.070684,Outerrace


In [19]:
df.info

<bound method DataFrame.info of             speed  vibration   z_score     Faults
0        4.217042   0.003062  0.291073       Ball
1        4.219672   0.004049  0.377935       Ball
2        4.221645   0.001418  0.146302       Ball
3        0.036346  -0.002199 -0.172193       Ball
4        0.036017  -0.003515 -0.288009       Ball
...           ...        ...       ...        ...
5667524  0.126758   0.010297  1.222905  Outerrace
5667542  3.131096  -0.001213  0.036795  Outerrace
5667560  1.329083  -0.000555  0.104573  Outerrace
5667571  3.977693  -0.000884  0.070684  Outerrace
5667578  0.155362   0.001747  0.341795  Outerrace

[870992 rows x 4 columns]>